In [1]:
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt

In [2]:
train_file='D:\\My Personal Documents\\Learnings\\Data Science\\Data Sets\\Genpact\\train.csv'
test_file='D:\\My Personal Documents\\Learnings\\Data Science\\Data Sets\\Genpact\\test.csv'
fulfilment_center='D:\\My Personal Documents\\Learnings\\Data Science\\Data Sets\\Genpact\\fulfilment_center_info.csv'
meal_info='D:\\My Personal Documents\\Learnings\\Data Science\\Data Sets\\Genpact\\meal_info.csv'

In [3]:
train_df= pd.read_csv(train_file)
test_df= pd.read_csv(test_file)
fulfilment_center_df= pd.read_csv(fulfilment_center)
meal_info_df= pd.read_csv(meal_info)

In [4]:
print(train_df.shape)
print(test_df.shape)
print(fulfilment_center_df.shape)
print(meal_info_df.shape)

(456548, 9)
(32573, 8)
(77, 5)
(51, 3)


In [5]:
train_test_df = train_df.append(test_df)

C:\Amrendra\Anaconda\lib\site-packages\pandas\core\frame.py:6201: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  sort=sort)


In [6]:
train_test_df.shape

(489121, 9)

In [7]:
train_test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 489121 entries, 0 to 32572
Data columns (total 9 columns):
base_price               489121 non-null float64
center_id                489121 non-null int64
checkout_price           489121 non-null float64
emailer_for_promotion    489121 non-null int64
homepage_featured        489121 non-null int64
id                       489121 non-null int64
meal_id                  489121 non-null int64
num_orders               456548 non-null float64
week                     489121 non-null int64
dtypes: float64(3), int64(6)
memory usage: 37.3 MB


In [8]:
train_test_fulfilment_df=pd.merge(train_test_df,fulfilment_center_df, how='inner', on='center_id')

In [9]:
train_test_fulfil_meal_df=pd.merge(train_test_fulfilment_df,meal_info_df,how='inner', on='meal_id')

In [10]:
train_test_fulfil_meal_df.shape

(489121, 15)

In [11]:
train_test_fulfil_meal_df.head()

,base_price,center_id,checkout_price,emailer_for_promotion,homepage_featured,id,meal_id,num_orders,week,city_code,region_code,center_type,op_area,category,cuisine
0,152.29,55,136.83,0,0,1379560,1885,177.0,1,647,56,TYPE_C,2.0,Beverages,Thai
1,152.29,55,135.83,0,0,1018704,1885,323.0,2,647,56,TYPE_C,2.0,Beverages,Thai
2,133.92,55,132.92,0,0,1196273,1885,96.0,3,647,56,TYPE_C,2.0,Beverages,Thai
3,134.86,55,135.86,0,0,1116527,1885,163.0,4,647,56,TYPE_C,2.0,Beverages,Thai
4,147.50,55,146.50,0,0,1343872,1885,215.0,5,647,56,TYPE_C,2.0,Beverages,Thai


In [12]:
train_test_cleaned=train_test_fulfil_meal_df.drop(labels=['city_code','center_id','meal_id','region_code'], axis=1)

In [13]:
train_test_cleaned['week1']=train_test_cleaned.week

In [14]:

def week_to_date(week):
    if(week<=53):
        return datetime.datetime.strptime("2013-W" + str(week) + '-0', "%Y-W%W-%w")
    elif(week>53 and week<=105):
        return datetime.datetime.strptime("2014-W" + str(week-53) + '-0', "%Y-W%W-%w")
    elif(week>105):
        return datetime.datetime.strptime("2015-W" + str(week-105) + '-0', "%Y-W%W-%w")
    
        
    

In [15]:
import datetime
train_test_cleaned.week=train_test_fulfil_meal_df.week.apply(week_to_date)

In [16]:
train_test_transformed=pd.get_dummies(train_test_cleaned, columns=['center_type','category','cuisine'])

In [17]:
train_test_transformed.head()

,base_price,checkout_price,emailer_for_promotion,homepage_featured,id,num_orders,week,op_area,week1,center_type_TYPE_A,...,category_Rice Bowl,category_Salad,category_Sandwich,category_Seafood,category_Soup,category_Starters,cuisine_Continental,cuisine_Indian,cuisine_Italian,cuisine_Thai
0,152.29,136.83,0,0,1379560,177.0,2013-01-13,2.0,1,0,...,0,0,0,0,0,0,0,0,0,1
1,152.29,135.83,0,0,1018704,323.0,2013-01-20,2.0,2,0,...,0,0,0,0,0,0,0,0,0,1
2,133.92,132.92,0,0,1196273,96.0,2013-01-27,2.0,3,0,...,0,0,0,0,0,0,0,0,0,1
3,134.86,135.86,0,0,1116527,163.0,2013-02-03,2.0,4,0,...,0,0,0,0,0,0,0,0,0,1
4,147.50,146.50,0,0,1343872,215.0,2013-02-10,2.0,5,0,...,0,0,0,0,0,0,0,0,0,1


In [18]:
train=train_test_transformed[train_test_transformed.week1<146]

In [19]:
test=train_test_transformed[train_test_transformed.week1>=146]

In [20]:
test.drop(labels=['num_orders','week1'],axis=1,inplace=True)


C:\Amrendra\Anaconda\lib\site-packages\pandas\core\frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [21]:
print(train.shape)
print(test.shape)

(456548, 30)
(32573, 28)


In [28]:
train_df=train[train.week1<=100].values
val_df= train[train.week1>100].values

In [27]:
type(train_df.values)

numpy.ndarray

In [23]:
train_y=train_df['num_orders'].values
train_x=train_df.drop(['num_orders','week1'],axis=1).values

In [24]:
val_y=val_df['num_orders'].values
val_x=val_df.drop(['num_orders','week1'],axis=1).values

# Reshape the data between -1 and 1 and to 3D
from sklearn.preprocessing import StandardScaler,MinMaxScaler
scaler = StandardScaler()
scaler = MinMaxScaler(feature_range=(-1, 1))
train_x = scaler.fit_transform(train_x)
val_x = scaler.fit_transform(val_x)

In [31]:
from math import sqrt
from numpy import split
from numpy import array
from pandas import read_csv
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import LSTM
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
 
# split a univariate dataset into train/test sets
def split_dataset(data):
	# split into standard weeks
	train, test = data[1:-328], data[-328:-6]
	# restructure into windows of weekly data
	train = array(split(train, len(train)/7))
	test = array(split(test, len(test)/7))
	return train, test
 
# evaluate one or more weekly forecasts against expected values
def evaluate_forecasts(actual, predicted):
	scores = list()
	# calculate an RMSE score for each day
	for i in range(actual.shape[1]):
		# calculate mse
		mse = mean_squared_error(actual[:, i], predicted[:, i])
		# calculate rmse
		rmse = sqrt(mse)
		# store
		scores.append(rmse)
	# calculate overall RMSE
	s = 0
	for row in range(actual.shape[0]):
		for col in range(actual.shape[1]):
			s += (actual[row, col] - predicted[row, col])**2
	score = sqrt(s / (actual.shape[0] * actual.shape[1]))
	return score, scores
 
# summarize scores
def summarize_scores(name, score, scores):
	s_scores = ', '.join(['%.1f' % s for s in scores])
	print('%s: [%.3f] %s' % (name, score, s_scores))
 
# convert history into inputs and outputs
def to_supervised(train, n_input, n_out=7):
	# flatten data
	data = train.reshape((train.shape[0]*train.shape[1], train.shape[2]))
	X, y = list(), list()
	in_start = 0
	# step over the entire history one time step at a time
	for _ in range(len(data)):
		# define the end of the input sequence
		in_end = in_start + n_input
		out_end = in_end + n_out
		# ensure we have enough data for this instance
		if out_end < len(data):
			X.append(data[in_start:in_end, :])
			y.append(data[in_end:out_end, 0])
		# move along one time step
		in_start += 1
	return array(X), array(y)
 
# train the model
def build_model(train, n_input):
	# prepare data
	train_x, train_y = to_supervised(train, n_input)
	# define parameters
	verbose, epochs, batch_size = 0, 50, 16
	n_timesteps, n_features, n_outputs = train_x.shape[1], train_x.shape[2], train_y.shape[1]
	# reshape output into [samples, timesteps, features]
	train_y = train_y.reshape((train_y.shape[0], train_y.shape[1], 1))
	# define model
	model = Sequential()
	model.add(LSTM(200, activation='relu', input_shape=(n_timesteps, n_features)))
	model.add(RepeatVector(n_outputs))
	model.add(LSTM(200, activation='relu', return_sequences=True))
	model.add(TimeDistributed(Dense(100, activation='relu')))
	model.add(TimeDistributed(Dense(1)))
	model.compile(loss='mse', optimizer='adam')
	# fit network
	model.fit(train_x, train_y, epochs=epochs, batch_size=batch_size, verbose=verbose)
	return model
 
# make a forecast
def forecast(model, history, n_input):
	# flatten data
	data = array(history)
	data = data.reshape((data.shape[0]*data.shape[1], data.shape[2]))
	# retrieve last observations for input data
	input_x = data[-n_input:, :]
	# reshape into [1, n_input, n]
	input_x = input_x.reshape((1, input_x.shape[0], input_x.shape[1]))
	# forecast the next week
	yhat = model.predict(input_x, verbose=0)
	# we only want the vector forecast
	yhat = yhat[0]
	return yhat
 
# evaluate a single model
def evaluate_model(train, test, n_input):
	# fit model
	model = build_model(train, n_input)
	# history is a list of weekly data
	history = [x for x in train]
	# walk-forward validation over each week
	predictions = list()
	for i in range(len(test)):
		# predict the week
		yhat_sequence = forecast(model, history, n_input)
		# store the predictions
		predictions.append(yhat_sequence)
		# get real observation and add to history for predicting the next week
		history.append(test[i, :])
	# evaluate predictions days for each week
	predictions = array(predictions)
	score, scores = evaluate_forecasts(test[:, :, 0], predictions)
	return score, scores
 
# load the new file
#dataset = read_csv('household_power_consumption_days.csv', header=0, infer_datetime_format=True, parse_dates=['datetime'], index_col=['datetime'])
# split into train and test
train=train_df
test=val_df
# evaluate model and get scores
n_input = 14
score, scores = evaluate_model(train, test, n_input)
# summarize scores
summarize_scores('lstm', score, scores)
# plot scores
days = ['sun', 'mon', 'tue', 'wed', 'thr', 'fri', 'sat']
pyplot.plot(days, scores, marker='o', label='lstm')
pyplot.show()
#Running the example fits the model and summarizes the performance on the test dataset.

C:\Amrendra\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


IndexError: tuple index out of range